In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, export_graphiz
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.processing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import model_selection

In [ ]:
%matplotlib inlines
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from eli5 import explain_weights, show_weights
from yellowbrick import ROCAuc
from yellowbrick.classifier import ClassificationReport

In [ ]:
print("Imported all libraries successfully")
print(os.listdir("Phys"))

In [ ]:
CV_N_REPEATS = 5
BINS = 10

In [ ]:
df = pd.read_csv("Vlagun_Phys_Years3.csv")
df.head()

In [ ]:
print("Shape of the dataset", df.shape)

In [ ]:
plt.figure()
plt.plot(df.Years,".")
plt.title("Years")
plt.xlabel("Index")
plt.show()

In [ ]:
plt.figure()
ax = sns.countplot(data=df, x="Years")
ax.set_title('Seaborn countplot of Years within the dataset')

In [ ]:
f, axes = plt.subplots(2,4,figsize=(15,15))
sns.set(style="whitegrid", palette="Set3", color_codes=True)
sns.boxplot(y='PSU', x='Years', data=df, orient='v', ax=axes[0,0])
sns.boxplot(y='O2', x='Years', data=df, orient='v', ax=axes[0,1])
sns.boxplot(y='temp', x='Years', data=df, orient='v', ax=axes[0,2])
sns.boxplot(y='SS', x='Years', data=df, orient='v', ax=axes[0,3])
sns.boxplot(y='DOC', x='Years', data=df, orient='v', ax=axes[1,0])
sns.boxplot(y='TPOC', x='Years', data=df, orient='v', ax=axes[1,1])
sns.boxplot(y='Windspeedinsitu', x='Years', data=df, orient='v', ax=axes[1,2])
sns.boxplot(y='Depth', x='Years', data=df, orient='v', ax=axes[1,3])

f.subplots_adjust(left=0.08, right=0.98, top=0.9, bottom=0.05, hspace=0.4, wspace=0.3)

plt.tight_layout()

In [ ]:
df_copy = df.copy(deep=True)
df_copy[['PSU', 'O2', 'temp', 'SS', 'DOC', 'TPOC', 'Windspeedinsitu', 'Depth']] = StandardScaler().fit_transform(df_copy[['PSU', 'O2', 'temp', 'SS', 'DOC', 'TPOC', 'Windspeedinsitu', 'Depth']])
print('Number of zero entries in each attribute:')
print(df_copy.isnull().sum())

In [ ]:
plt.figure(figsize=((15,15)))
corr=df_copy.corr()
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot=True, cmap=plt.cm.Reds)
plt.title('Correlation Heatmap', fontsize=20)
plt.show()

In [ ]:
plt.figure()
sns.pairplot(data=df_copy, hue='Years', diag_kind='kde', palette='deep')

In [ ]:
X = df.iloc[:,0:8].values
y = df.iloc[:,8].values

seed = 7
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

In [ ]:
dum = DummyClassifier(strategy='most_frequent')
dum.fit(X_train, y_train)
score = dum.score(X_test, y_test)
print("Dummy Classifier accuracy: %.2f%%" % (score*100.0))

In [ ]:
strategy = 'most_frequent'
scores = cross_val_score(dum, X,y, cv=RepeatedKFold(n_repeats=CV_N_REPEATS, n_splits=10), scoring=None)
scores_dummy = scores.copy()
score_line = 'Scores (Accuracy) mean={0:.2f} +/- {1:.2f}'.format(scores.mean(), scores.std())
plt.figure(figsize=(7,7))
fig, ax = plt.subplots()
pd.Series(scores).hist(ax=ax, bins=BINS)
ax.set_title(f"RepeatedKFold ({len(scores)} folds) with DummyClassifier ({strategy}) \n + {score_line}")
ax.set_xlabel('Score')
ax.set_ylabel('Frequency')

In [ ]:
def plot_tree_graph(model, columns, class_names):
    dot_data = export_graphviz(model, feature_names=columns, class_names=class_names)
    graph = graphviz.Source(dot_data)
    return graph

In [ ]:
def confusion_mat(Y_pred, Y_test):
    plt.figure()
    cm = confusion_matrix(Y_pred, Y_test)
    sns.heatmap(cm,annot=True, fmt='g')
    plt.title('Confusion matrix')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
dt = DecisionTreeClassifier(random_state=1, max_depth=3)
dt = dt.fit(X_train, y_train)
dt_scores = cross_val_score(dt, X,y, cv=RepeatedStratifiedKFold(n_repeats=CV_N_REPEATS))
print(f"Accuracy mean = {dt_scores.mean()} +/- {dt_scores.std()}")


In [ ]:
bag = BaggingClassifier(n_estimators = 100, oob_score=True, random_state=1)
bag=bag.fit(X_train, y_train)
bag_scores = cross_val_score(bag, X,y, cv=RepeatedStratifiedKFold(n_repeats=CV_N_REPEATS))
print(f"Accuracy mean = {bag_scores.mean()} +/- {bag_scores.std()}")
print("Out of bag score = %.2f" % bag.oob_score_*100)

In [ ]:
num_estimators = 100
rf = RandomForestClassifier(n_estimators=num_estimators, random_state=1)
rf = rf.fit(X_train, y_train)
rf_scores = rf.score(X_test, y_test)
print("Random Forest accuracy: %.2f%%" % (rf_scores*100.0))
y_pred = rf.predict(X_test)
confusion_mat(y_pred, y_test)

In [ ]:
feature_names = X_train.columns.values
show_weights(rf, feature_names= feature_names)

In [ ]:
scores = cross_val_score(rf, X,y, cv=RepeatedStratifiedKFold(n_repeats=CV_N_REPEATS))
scores_rf = scores.copy()
print(f"Scores mean = {scores.mean()}, stddev = {scores.std()}")
score_line = 'Scores (Accuracy) mean={0:.2f} +/- {1:.2f}'.format(scores.mean(), scores.std())
plt.figure(figsize=(7,7))
fig, ax = plt.subplots()
pd.Series(scores).hist(ax=ax, bins=BINS)
ax.set_title(f"RepeatedKFold ({len(scores)} folds) with RandomForestClassifier + {score_line}")
ax.set_xlabel('Score')
ax.set_ylabel('Frequency')


In [ ]:
from sklearn.metrics import mean_squared_error
params={'n_estimators':[500], "learning_rate": 0.01, "max_depth": 5, "loss":'deviance'}
gbm = GradientBoostingClassifier(**params)
gbm.fit(X_train, y_train)

In [ ]:
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(gbm.staged_predict(X_test)):
    test_score[i] = gbm.loss_(y_test, y_pred)
plt.figure(figsize=(7,7))
plt.subplot(1,2,1)
plt.title('GBM Deviance w.r.t Number of Estimators')
plt.plot(np.arange(params['n_estimators']) + 1, gbm.train_score_, 'b-',
            label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
            label='Test Set Deviance')
plt.legend(loc='best')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

In [ ]:
params={"n_estimators": 100, "max_depth": 4, "learning_rate": 0.01, "loss": 'deviance'}
gbm = GradientBoostingClassifier(**params)
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_test)

gbm_score = accuracy_score(y_test, y_pred)
print("GBM accuracy: %.2f%%" % (gbm_score*100.0))
confusion_mat(y_pred, y_test)

In [ ]:
feature_importance = gbm.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.figure(figsize=(7,7))
plt.subplot(1,2,2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, df.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
from xgboost import XGBClassifier, plot_importance, to_graphviz

param = {'max_depth':10, 'eta':0.8, 'subsample':1, 'objective':'binary:logistic', 'n_estimators':1000, 'learning_rate':0.001}
xgb = XGBClassifier(**param)
xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict(X_test)
xgb_score = accuracy_score(y_test, y_pred)
print("XGB accuracy: %.2f%%" % (xgb_score*100.0))
confusion_mat(y_pred, y_test)

In [ ]:
plt.figure()
plot_importance(xgb, title='Feature importance from XGboost model')
plt.show()

In [ ]:
from sklearn.preprocesing import OneHotEncoder, LabEncoder
from sklearn import tree

In [ ]:
data = pd.read_csv('Vlagun_Phys_Year3.csv')
data.dropna(inplace=True)
X = pd.get_dummies(data)
X.drop(['Years'], inplace=True, axis=1)
y = data['Years']
data.head()

In [ ]:
X = df.iloc[:,0:8].values
y = df.iloc[:,8].values
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

In [ ]:
model = XGBClassifier(n_estimators = 1000, max_depth=10, learning_rate=0.001)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
model_score = accuracy_score(y_test, y_pred)
print("XGB accuracy: %.2f%%" % (model_score*100.0))
confusion_mat(y_pred, y_test)

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

In [ ]:
i=0
shap.force_plot(explainer.expected_value[i], shap_values[i], features = X.iloc[i], features_names = X.columns)

In [ ]:
i= 109
shap.force_plot(explainer.expected_value[i], shap_values[i], features = X.iloc[i], features_names = X.columns)

In [ ]:
shap.summary_plot(shap_values, features = X, feature_names = X.columns, show=False)
plt.savefig('plot.png', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
shap.summary_plot(shap_values, features = X, feature_names = X.columns, show=False, plot_type = 'bar')
plt.savefig('plot2.png', dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
shap.dependence_plot('Windspeedinsitu', shap_values, features = X, interaction_index = 'DOC')

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[:121,:], show=False, features=X.iloc[:121,:])